<a href="https://colab.research.google.com/github/vincimech010233/QuantumComputingJourney-/blob/main/HybridQuantumResNetClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio
!pip install pennylane

In [10]:
import torch
import torchvision
import torch.nn as nn
import pennylane as qml
from pennylane import numpy as pnp

# Configurar el dispositivo CUDA si está disponible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Configurar el dispositivo cuántico de PennyLane
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Definición del circuito cuántico
@qml.qnode(dev, interface='torch')
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Número de capas en el circuito cuántico
n_layers = 2
weight_shapes = {"weights": (n_layers, n_qubits, 3)}

# Clase para el circuito cuántico
class QuantumLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(n_layers, n_qubits, 3))
        self.quantum_circuit = quantum_circuit

    def forward(self, x):
        return self.quantum_circuit(x, self.weights)

# Integrar el circuito cuántico con una red ResNet preentrenada
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        # Actualización para evitar el aviso de 'pretrained'
        self.resnet = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        self.resnet.fc = nn.Linear(512, n_qubits)  # Ajuste para coincidir con la entrada cuántica
        self.quantum_layer = QuantumLayer()
        self.fc = nn.Linear(n_qubits, 2)  # Capa final para clasificación

    def forward(self, x):
        x = self.resnet(x)  # Procesamiento clásico
        x = self.quantum_layer(x)  # Procesamiento cuántico
        x = self.fc(x)  # Capa de clasificación
        return torch.log_softmax(x, dim=1)

# Crear una instancia del modelo híbrido y mover al dispositivo adecuado
model = HybridModel().to(device)

In [11]:
print(model)

HybridModel(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_